<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=182267883" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/luminlab/ber-curation?scriptVersionId=182257992" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Details of Steps can be found on Google Slides via:¶

https://docs.google.com/presentation/d/1sb3QkXiYooHqi3p-tkGVUqwqFKd-601_pzU96W1drw0/edit?usp=sharing

In [1]:
from kaggle_secrets import UserSecretsClient
wandb_key_label = "WANDB_KEY"
wandb_key= UserSecretsClient().get_secret(wandb_key_label)

In [2]:
import git
git.Repo.clone_from('https://github.com/Lumin-Lab/BerCuration', '/kaggle/working/scarf')

<git.repo.base.Repo '/kaggle/working/scarf/.git'>

In [ ]:
!pip install -r /kaggle/working/scarf/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 93.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 92.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.3/352.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 93.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 87.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
def save_csv_file(df, path):
    dir_name = os.path.dirname(path)
    if dir_name:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    df.to_csv(path, index=False)

In [ ]:
import pandas
df = pd.read_csv("/kaggle/input/ber-stratified-samples/BER_stratified_sample.csv")

In [ ]:
n_splits = 5
output_dir = "/kaggle/working/output"
config_dir="/kaggle/working/scarf/configs"
scarf_model_name = "scarf"
mlp_model_name = "mlp"

In [ ]:
from sklearn.model_selection import KFold
import os
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(df)):
    save_csv_file(df.iloc[train_index], f"{output_dir}/split_{i+1}/raw_train.csv")
    save_csv_file(df.iloc[test_index], f"{output_dir}/split_{i+1}/raw_test.csv")

In [ ]:
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir "/kaggle/working/scarf/configs" \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/raw_train.csv" \
      --output_csv_name "processed_train" \
      --is_train
    """
    os.system(command)
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir "/kaggle/working/scarf/configs" \
      --output_dir "{output_dir}/split_{i+1}" \
      --data_path "{output_dir}/split_{i+1}/raw_test.csv" \
      --output_csv_name "processed_test"
    """
    os.system(command)

**Train the SCARF Encoder on the Train Dataset**

In [ ]:
scarf_batch_size = 32
scarf_epochs = 25
scarf_lr = 3e-5
scarf_emb_dim = 32
scarf_encoder_depth = 3
scarf_corruption_rate=0.3

In [ ]:
import os
"""The trained scarf model is saved in 
/kaggle/working/output/split_{split}/scarf.pt if you run the following command:
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/run_scarf.py\
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}" \
      --train_data_path="{output_dir}/split_{i+1}/processed_train.csv"\
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name="{scarf_model_name}" \
      --corruption_rate={scarf_corruption_rate} \
      --wandb_project_name='SCARF_Project' \
      --wandb_entity='urbancomp' \
      --wandb_key='{wandb_key}'
    """

    os.system(command)

**Obtain the SCARF embeddings for different train splits, and save the result**

In [ ]:
"""The generated embeddings are saved as a NumPy array in 
/kaggle/working/output/split_{split}/train.npy if you run the following command:
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_scarf_embedding.py \
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}" \
      --data_path="{output_dir}/split_{i+1}/processed_train.csv" \
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name={scarf_model_name} \
      --corruption_rate={scarf_corruption_rate} \
      --embedding_save_name="train"
    """

    os.system(command)

**Obtain the SCARF embeddings for different test splits, and save the result**

In [ ]:
"""The generated embeddings are saved as a NumPy array in 
/kaggle/working/output/split_{split}/test.npy if you run the following command:
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_scarf_embedding.py \
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}" \
      --data_path="{output_dir}/split_{i+1}/processed_test.csv" \
      --batch_size={scarf_batch_size} \
      --epochs={scarf_epochs} \
      --lr={scarf_lr} \
      --emb_dim={scarf_emb_dim} \
      --encoder_depth={scarf_encoder_depth} \
      --model_name={scarf_model_name} \
      --corruption_rate={scarf_corruption_rate} \
      --embedding_save_name="test"
    """

    os.system(command)

**Filter outliers based on scarf embeddings**

In [ ]:
threshold = 0.3

In [ ]:
label_map = {
    0: 'A1',
    1: 'A2',
    2: 'A3',
    3: 'B1',
    4: 'B2',
    5: 'B3',
    6: 'C1',
    7: 'C2',
    8: 'C3',
    9: 'D1',
    10: 'D2',
    11: 'E1',
    12: 'E2',
    13: 'F',
    14: 'G'
}

In [ ]:
import matplotlib.pyplot as plt
def plot_data(data, labels, circles, title, alpha=1.0):
    
    colormap = plt.cm.tab20
    colors = {str(label): colormap(i) for i, label in enumerate(set(labels))}

    plt.figure(figsize=(14, 5))
    done = set()
    for i in range(0,len(data)):
        lab = str(labels[i])
        label_name = label_map[int(lab)]
        if label_name in done:
            label = ""
        else:
            label = label_name
            done.add(label_name)
        plt.scatter(data[i, 0], data[i, 1],  c=colors[lab], s=30,alpha=0.6, 
                    label = label)
    for i in circles:
        plt.plot(
            data[i][0],
            data[i][1],
            "o",
            markerfacecolor="none",
            markeredgecolor="red",
            markersize=14,
            markeredgewidth=2.5,
            alpha=alpha
        )
    _ = plt.title(title, fontsize=25)
    plt.legend()

In [ ]:
from cleanlab.outlier import OutOfDistribution
import numpy as np
from sklearn.decomposition import PCA

for i in range(n_splits):
    ood = OutOfDistribution()
    train_emb = np.load(f"{output_dir}/split_{i+1}/train.npy")
    test_emb = np.load(f"{output_dir}/split_{i+1}/test.npy")
    ood.fit_score(features=train_emb)
    ood_train_feature_scores = ood.score(features=train_emb)
    ood_test_feature_scores = ood.score(features=test_emb)
    train_outliers_idx = np.where(ood_train_feature_scores  < threshold)[0]
    test_outliers_idx = np.where(ood_test_feature_scores  < threshold)[0]
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_train.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/raw_test.csv")
    save_csv_file(train_df.iloc[train_outliers_idx], f"{output_dir}/split_{i+1}/cleanlab/train_outliers.csv")
    save_csv_file(test_df.iloc[test_outliers_idx], f"{output_dir}/split_{i+1}/cleanlab/test_outliers.csv")
    save_csv_file(train_df[~train_df.index.isin(train_outliers_idx)], f"{output_dir}/split_{i+1}/cleanlab/train_removed_outliers.csv")
    save_csv_file(test_df[~test_df.index.isin(test_outliers_idx)], f"{output_dir}/split_{i+1}/cleanlab/test_removed_outliers.csv")
    
    pca = PCA(n_components=2)
    process_train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train.csv")
    
    process_test_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_test.csv")
    
    pca.fit(process_train_df.drop(columns=["EnergyRating"]))
    low_dim_train_features = pca.transform(process_train_df.drop(columns=["EnergyRating"]))
    low_dim_test_features = pca.transform(process_test_df.drop(columns=["EnergyRating"]))
    
    colormap = plt.cm.tab20
    colors = [colormap(i) for i in range(len(label_map))]
    
    plot_data(low_dim_train_features, process_train_df["EnergyRating"], 
              train_outliers_idx, 
              title = f"Split {i+1}: Outliers in Training Set",
             )
    plot_data(low_dim_test_features, process_test_df["EnergyRating"], 
              test_outliers_idx, 
              title = f"Split {i+1}: Outliers in Test Set",
             )

In [ ]:
"""Process train and test sets after their outliers are removed.
"""
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir "/kaggle/working/scarf/configs" \
      --output_dir "{output_dir}/split_{i+1}/cleanlab" \
      --data_path "{output_dir}/split_{i+1}/cleanlab/train_removed_outliers.csv" \
      --output_csv_name "processed_train" \
      --is_train
    """
    os.system(command)
    command = f"""
    python /kaggle/working/scarf/get_processed_dataset.py \
      --config_dir "/kaggle/working/scarf/configs" \
      --output_dir "{output_dir}/split_{i+1}/cleanlab" \
      --data_path "{output_dir}/split_{i+1}/cleanlab/test_removed_outliers.csv" \
      --output_csv_name "processed_test"
    """
    os.system(command)

**Train the MLP classifier on the datasets**

In [ ]:
mlp_batch_size = 32
mlp_epochs = 25
mlp_lr = 0.00003
mlp_dropout= 0.1

**Before outlier removal**

In [ ]:
for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/run_mlp.py \
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}/"\
      --train_data_path="{output_dir}/split_{i+1}/processed_train.csv" \
      --test_data_path="{output_dir}/split_{i+1}/cleanlab/processed_test.csv" \
      --batch_size={mlp_batch_size} \
      --epochs={mlp_epochs} \
      --lr={mlp_lr} \
      --model_name={mlp_model_name} \
      --wandb_project_name "test" \
      --wandb_entity "urbancomp" \
      --wandb_key {wandb_key} \
      --hidden_layer 256 128 64 32 16 \
      --dropout={mlp_dropout} 
    """

    os.system(command)

**After outlier removal**

In [ ]:

for i in range(n_splits):
    command = f"""
    python /kaggle/working/scarf/run_mlp.py \
      --config_dir={config_dir} \
      --output_dir="{output_dir}/split_{i+1}/cleanlab"\
      --train_data_path="{output_dir}/split_{i+1}/cleanlab/processed_train.csv" \
      --test_data_path="{output_dir}/split_{i+1}/cleanlab/processed_test.csv" \
      --batch_size={mlp_batch_size} \
      --epochs={mlp_epochs} \
      --lr={mlp_lr} \
      --model_name={mlp_model_name} \
      --wandb_project_name "test" \
      --wandb_entity "urbancomp" \
      --wandb_key {wandb_key} \
      --hidden_layer 256 128 64 32 16 \
      --dropout={mlp_dropout} 
    """

    os.system(command)

**Train the random forest on the datasets**

In [ ]:
max_depth = 6

**Before outlier removal**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
for i in range(n_splits):
    clf = RandomForestClassifier(max_depth=max_depth, random_state=42)
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/processed_train.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/cleanlab/processed_test.csv")
    clf.fit(train_df.drop(columns=["EnergyRating"]), train_df["EnergyRating"])
    acc = clf.score(test_df.drop(columns=["EnergyRating"]), test_df["EnergyRating"],
             )
    predict = clf.predict(test_df.drop(columns=["EnergyRating"]))
    f1 = f1_score(test_df["EnergyRating"], predict,average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")

**After outlier removal**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
for i in range(n_splits):
    clf = RandomForestClassifier(max_depth=max_depth, random_state=42)
    train_df = pd.read_csv(f"{output_dir}/split_{i+1}/cleanlab/processed_train.csv")
    test_df = pd.read_csv(f"{output_dir}/split_{i+1}/cleanlab/processed_test.csv")
    clf.fit(train_df.drop(columns=["EnergyRating"]), train_df["EnergyRating"])
    acc = clf.score(test_df.drop(columns=["EnergyRating"]), test_df["EnergyRating"],
             )
    predict = clf.predict(test_df.drop(columns=["EnergyRating"]))
    f1 = f1_score(test_df["EnergyRating"], predict,average='macro')
    print(f"Split: {i+1}, Accuracy: {acc}, F1 Score: {f1}")